In [1]:
%load_ext autoreload
%load_ext autotime

time: 265 µs (started: 2024-01-15 19:43:45 -08:00)


In [2]:
import os
import sys
import warnings
wisio_dir = os.path.abspath("../../")
sys.path.append(wisio_dir)
warnings.filterwarnings('ignore')

time: 1.02 ms (started: 2024-01-15 19:43:45 -08:00)


In [3]:
import csv
import dask
import dask.dataframe as dd
import json
import numpy as np
import pandas as pd
import sqlite3
print('pandas', pd.__version__)
print('dask', dask.__version__)
print('numpy', np.__version__)

pandas 2.1.2
dask 2023.9.3
numpy 1.24.3
time: 2.2 s (started: 2024-01-15 19:43:52 -08:00)


## Main View

In [103]:
import functools as ft
from glob import glob
from pathlib import Path

df = None
# df_dir = '/g/g91/izzet/projects/wisio/.wisio/repeats/recorder_montage_32_lsf'
folder = '../../.wisio/repeats'

dfs = []

for file_dir in os.listdir(folder):
    df = None

    col_len = 0
    for file_name in os.listdir(f"{folder}/{file_dir}"):
        if file_name.endswith('.csv') and '_' not in file_name:
            col_len = col_len + 1
            if df is None:
                df = pd.read_csv(f"{folder}/{file_dir}/{file_name}")
            else:
                df = df.merge(pd.read_csv(
                    f"{folder}/{file_dir}/{file_name}"), on=['type', 'value'], how='outer')

    if df is not None:

        df_dict = {}
        stds = {}
        for ix, row in df.iterrows():
            df_dict[ix] = {}
            df_dict[ix]['type'] = row['type']
            df_dict[ix]['value'] = row['value']

            is_numeric = True
            try:
                try:
                    values = np.array([int(row[col])
                                      for col in df.columns[-col_len:]])
                except:
                    values = np.array([float(row[col])
                                      for col in df.columns[-col_len:]])
            except ValueError:
                values = np.array([row[col] for col in df.columns[-col_len:]])
                is_numeric = False

            if is_numeric:
                stds[row['type']] = stds.get(row['type'], {})
                stds[row['type']][row['value']] = stds[row['type']].get(
                    row['value'], {})
                stds[row['type']][row['value']][df.columns[-1]] = values.std()
                df_dict[ix][df.columns[-1]] = values.mean()
            else:
                df_dict[ix][df.columns[-1]] = values[0]

        i = 0
        for t in stds:
            for v in stds[t]:
                i = i + 1
                ix = len(df_dict) + i
                df_dict[ix] = {}
                df_dict[ix]['type'] = t
                df_dict[ix]['value'] = f"{v}_std"
                df_dict[ix][df.columns[-1]] = stds[t][v][df.columns[-1]]

        dfs.append(pd.DataFrame.from_dict(df_dict, orient='index'))

merged_df = ft.reduce(pd.merge, dfs)

merged_df

# stds

,type,value,LBANNJag_y,Montage,CM1,MummiWemul,HACC,MontagePegasus,Flash,LBANNCosmoflow_y
0,characteristics,complexity,5.468182,7.20474,8.590904,10.327057,7.962778,10.278549,7.941626,12.965239
1,characteristics,io_time,8.065208,8.474974,4.1151,9271.347713,10.367438,3.739198,1.988732,134.055023
2,characteristics,job_time,194355120.0,175.248932,667.808838,3718.616699,32.742073,433.145325,53.975018,12023159.0
3,characteristics,num_apps,1.0,5.0,1.0,1.0,1.0,10.0,1.0,1.0
4,characteristics,num_files,4.0,1937.0,775.0,74541.0,2561.0,19680.0,1581.0,49521.0
...,...,...,...,...,...,...,...,...,...,...
184,timings,time__compute_hlm_std,7.990834,3.164125,0.253127,1.52356,1.122054,3.780332,0.819162,6.988964
185,timings,time__compute_main_view_std,0.03944,0.107508,0.004044,0.034722,0.021293,0.059928,0.01098,0.042457
186,timings,time__compute_perspectives_std,0.346544,0.024326,0.080753,0.191356,0.116285,0.147244,0.163206,1.28185
187,timings,time__detect_bottlenecks_std,0.963709,0.04602,0.032877,0.011413,0.023179,0.173824,0.025041,0.026543


time: 333 ms (started: 2023-12-26 01:29:52 -08:00)


In [104]:
merged_df.to_csv('main_views.csv', encoding='utf-8')

time: 5.23 ms (started: 2023-12-26 01:30:14 -08:00)


In [55]:
df_dict = {}
for ix, row in df.iterrows():
    df_dict[ix] = {}
    df_dict[ix]['type'] = row['type']
    df_dict[ix]['value'] = row['value']

    is_numeric = True
    try:
        try:
            values = np.array([int(row[col]) for col in df.columns[-3:]])
        except:
            values = np.array([float(row[col]) for col in df.columns[-3:]])
    except ValueError:
        values = np.array([row[col] for col in df.columns[-3:]])
        is_numeric = False

    if is_numeric:
        df_dict[ix][df.columns[-1]] = values.mean()
    else:
        df_dict[ix][df.columns[-1]] = values[0]


pd.DataFrame.from_dict(df_dict, orient='index')

,type,value,Montage
0,characteristics,complexity,7.20474
1,characteristics,io_time,8.474974
2,characteristics,job_time,175.248932
3,characteristics,num_apps,5.0
4,characteristics,num_files,1937.0
...,...,...,...
92,timings,time__compute_hlm,39.300131
93,timings,time__compute_main_view,1.450593
94,timings,time__compute_perspectives,7.566197
95,timings,time__detect_bottlenecks,3.236037


time: 30.8 ms (started: 2023-12-25 21:15:52 -08:00)


In [64]:
df = None
main_view_dir = 'main-view-results'
for file_name in os.listdir(main_view_dir):
    if df is None:
        df = pd.read_csv(f"{main_view_dir}/{file_name}")
    else:
        df = df.merge(pd.read_csv(
            f"{main_view_dir}/{file_name}"), on=['type', 'value'], how='outer')

df

,type,value,CM1,Flash,HACC,Montage,MontagePegasus,MummiWemul,LBANNCosmoflow,LBANNJag
0,characteristics,complexity,8.590904222529606,7.94162558639499,7.962777579458518,7.204740146468978,10.27854880705543,10.32705717537284,12.965239004579356,5.468181853373804
1,characteristics,io_time,4.1151000122494,1.988731704874965,10.367437948642419,8.474973793299455,3.7391976846277544,9271.34771273648,134.05502345071136,8.065207895234778
2,characteristics,job_time,667.808837890625,53.97501754760742,32.74207305908203,175.24893188476562,433.14532470703125,3718.61669921875,12023159.0,194355120.0
3,characteristics,num_apps,1.0,1.0,1.0,5.0,10.0,1.0,1.0,1.0
4,characteristics,num_files,775.0,1581.0,2561.0,1937.0,19680.0,74541.0,49521.0,4.0
...,...,...,...,...,...,...,...,...,...,...
92,timings,time__compute_hlm,3.750587295740843,16.577061087999027,7.482202622049954,36.581391375977546,53.207441051024944,37.2190774360206,947.5043942820048,1846.7460069709923
93,timings,time__compute_main_view,1.3570610210299492,2.746419468021486,1.4695786730153486,1.3529689729912207,2.55050134396879,10.512798758980352,14.777734991977923,1.340656137035694
94,timings,time__compute_perspectives,6.23406346142292,6.239699762954842,6.326297864026856,6.24541637103539,6.286926453001797,6.738769157032948,7.3268226200016215,7.799504827999044
95,timings,time__detect_bottlenecks,6.550481762737036,9.16915659501683,6.943220883025788,6.681236644042656,8.953589458018541,9.473824601038359,24.08344194100937,6.726484668964986


time: 31.8 ms (started: 2023-12-23 18:24:41 -08:00)


In [65]:
df.to_csv('main_views.csv')

time: 2.62 ms (started: 2023-12-23 18:24:42 -08:00)


## Perspectives

In [25]:
from glob import glob
from pathlib import Path

for folder in glob('../../.wisio/*_view_types'):
    df = None
    for file_name in os.listdir(folder):
        if file_name.endswith('.csv') and not '_' in file_name:
            if df is None:
                df = pd.read_csv(f"{folder}/{file_name}")
            else:
                df = df.merge(pd.read_csv(
                    f"{folder}/{file_name}"), on=['type', 'value'], how='outer')

    df = df.set_index(['type', 'value'])

    perspectives = []
    for i, x in enumerate(df.loc[('counts', 'num_perspectives')].copy()):
        perspective = int(float(x))
        df.loc[('counts', 'num_perspectives'), df.columns[i]] = perspective
        perspectives.append(perspective)

    df = df.sort_values(('counts', 'num_perspectives'), ascending=True, axis=1)

    perspectives_str = list(map(str, sorted(perspectives)))
    col_names = {col: perspectives_str[i] for i, col in enumerate(df.columns)}

    df = df.rename(columns=col_names)

    csv_name = Path(folder).name.replace('view_types', 'perspectives')

    df.to_csv(f"perspectives-merged/{csv_name}.csv")

    print(f"Saved {csv_name}.csv")

Saved recorder_cm1_perspectives.csv
Saved recorder_hacc_perspectives.csv
Saved recorder_montage_perspectives.csv
Saved recorder_mummi_wemul_perspectives.csv
Saved recorder_flash_perspectives.csv
Saved recorder_lbann_jag_perspectives.csv
Saved recorder_montage_pegasus_perspectives.csv
Saved recorder_lbann_cosmoflow_perspectives.csv
time: 103 ms (started: 2023-12-28 15:05:57 -08:00)


## Scales

In [11]:
from glob import glob
from pathlib import Path

for folder in glob('../../.wisio/scales/*'):
    
    df = None
    for i, file_name in enumerate(os.listdir(folder)):
        if file_name.endswith('.csv') and '_' not in file_name:
            if df is None:
                df = pd.read_csv(f"{folder}/{file_name}")
            else:
                df = df.merge(pd.read_csv(
                    f"{folder}/{file_name}"), on=['type', 'value'], how='outer', suffixes=('', f"{i}"))

    if df is None:
        continue

    df = df.set_index(['type', 'value'])

    label = ('setup', 'num_workers')

    for i, x in enumerate(df.loc[label].copy()):
        df.loc[label, df.columns[i]] = float(x)

    csv_name = Path(folder).name

    df.sort_values(label, ascending=True,
                   axis=1).to_csv(f"scale-merged/{csv_name}_scales.csv")

    print(f"Saved {csv_name}.csv")

Saved recorder_montage.csv
Saved recorder_flash.csv
Saved recorder_cm1.csv
Saved recorder_montage_pegasus.csv
Saved recorder_hacc.csv
Saved recorder_mummi_wemul.csv
time: 89.2 ms (started: 2023-12-26 12:04:01 -08:00)


## Thresholds

### Metric Thresholds

In [4]:
from glob import glob
from pathlib import Path

for folder in glob('../../.wisio/*_thresholds'):
    df = None
    for i, file_name in enumerate(os.listdir(folder)):
        if file_name.endswith('.csv') and not file_name.endswith('_timings.csv'):
            if df is None:
                df = pd.read_csv(f"{folder}/{file_name}")
            else:
                df = df.merge(pd.read_csv(
                    f"{folder}/{file_name}"), on=['type', 'value'], how='outer', suffixes=('', f"{i}"))

    if df is None:
        continue

    df = df.set_index(['type', 'value'])

    label = ('setup', 'metric_threshold')

    for i, x in enumerate(df.loc[label].copy()):
        df.loc[label, df.columns[i]] = float(x)

    csv_name = Path(folder).name

    df.sort_values(label, ascending=False,
                   axis=1).to_csv(f"{csv_name}.csv")

    print(f"Saved {csv_name}.csv")

Saved recorder_cm1_thresholds.csv
Saved recorder_flash_thresholds.csv
Saved recorder_hacc_thresholds.csv
Saved recorder_montage_thresholds.csv
Saved recorder_montage_pegasus_thresholds.csv
time: 167 ms (started: 2023-12-23 22:46:54 -08:00)


### Slope Thresholds

In [28]:
from glob import glob
from pathlib import Path

for folder in glob('../../.wisio/slope_thresholds/*'):

    df = None
    for i, file_name in enumerate(os.listdir(folder)):
        if file_name.endswith('.csv') and '_' not in file_name:
            if df is None:
                df = pd.read_csv(f"{folder}/{file_name}")
            else:
                df = df.merge(pd.read_csv(
                    f"{folder}/{file_name}"), on=['type', 'value'], how='outer', suffixes=('', f"{i}"))

    if df is None:
        continue

    df = df.set_index(['type', 'value'])

    label = ('setup', 'slope_threshold')

    slopes = []
    for i, x in enumerate(df.loc[label].copy()):
        slope = int(float(x))
        df.loc[label, df.columns[i]] = slope
        slopes.append(slope)

    df = df.sort_values(label, ascending=True, axis=1)

    slopes_str = list(map(str, sorted(slopes)))
    col_names = {col: slopes_str[i] for i, col in enumerate(df.columns)}

    df = df.rename(columns=col_names)

    csv_name = Path(folder).name

    df.to_csv(f"slope-merged/{csv_name}.csv")

    print(f"Saved {csv_name}.csv")

Saved recorder_flash.csv
Saved recorder_hacc.csv
Saved recorder_montage_pegasus.csv
Saved recorder_cm1.csv
Saved recorder_lbann_cosmoflow.csv
Saved recorder_lbann_jag.csv
Saved recorder_mummi_wemul.csv
Saved recorder_montage.csv
time: 166 ms (started: 2023-12-28 21:57:39 -08:00)


In [5]:
from glob import glob
from pathlib import Path

for folder in glob('../../.wisio/slope_thresholds/*'):

    df = None
    for i, file_name in enumerate(os.listdir(folder)):
        if file_name.endswith('.csv') and '_' not in file_name:
            if df is None:
                df = pd.read_csv(f"{folder}/{file_name}")
                break
            else:
                df = df.merge(pd.read_csv(
                    f"{folder}/{file_name}"), on=['type', 'value'], how='outer', suffixes=('', f"{i}"))

    if df is None:
        continue

    df = df.set_index(['type', 'value'])

    label = ('setup', 'slope_threshold')

    slopes = []
    for i, x in enumerate(df.loc[label].copy()):
        slope = int(float(x))
        df.loc[label, df.columns[i]] = slope
        slopes.append(slope)

    df = df.sort_values(label, ascending=True, axis=1)

    slopes_str = list(map(str, sorted(slopes)))
    col_names = {col: slopes_str[i] for i, col in enumerate(df.columns)}

    df = df.rename(columns=col_names)

    break

df

15
type            value                                         
characteristics complexity                    7.94162558639499
                io_time                      1.988731704874965
                job_time                     53.97501754760742
                num_apps                                   1.0
                num_files                               1581.0
...                                                        ...
timings         time__compute_hlm           15.239039059961215
                time__compute_main_view     2.7886829250492156
                time__compute_perspectives   5.691171610960737
                time__detect_bottlenecks    2.4482154350262135
                time__read_traces                            0

[150 rows x 1 columns]

time: 18.1 ms (started: 2024-01-15 19:45:25 -08:00)


In [6]:
df = df.reset_index()
df

,type,value,15
0,characteristics,complexity,7.94162558639499
1,characteristics,io_time,1.988731704874965
2,characteristics,job_time,53.97501754760742
3,characteristics,num_apps,1.0
4,characteristics,num_files,1581.0
...,...,...,...
145,timings,time__compute_hlm,15.239039059961215
146,timings,time__compute_main_view,2.7886829250492156
147,timings,time__compute_perspectives,5.691171610960737
148,timings,time__detect_bottlenecks,2.4482154350262135


time: 10.9 ms (started: 2024-01-15 19:46:11 -08:00)


In [7]:
df['key'] = df['type'] + '_' + df['value']
df

,type,value,15,key
0,characteristics,complexity,7.94162558639499,characteristics_complexity
1,characteristics,io_time,1.988731704874965,characteristics_io_time
2,characteristics,job_time,53.97501754760742,characteristics_job_time
3,characteristics,num_apps,1.0,characteristics_num_apps
4,characteristics,num_files,1581.0,characteristics_num_files
...,...,...,...,...
145,timings,time__compute_hlm,15.239039059961215,timings_time__compute_hlm
146,timings,time__compute_main_view,2.7886829250492156,timings_time__compute_main_view
147,timings,time__compute_perspectives,5.691171610960737,timings_time__compute_perspectives
148,timings,time__detect_bottlenecks,2.4482154350262135,timings_time__detect_bottlenecks


time: 11.9 ms (started: 2024-01-15 19:46:31 -08:00)


In [8]:
df = df.drop(columns=['type', 'value'])
df

,15,key
0,7.94162558639499,characteristics_complexity
1,1.988731704874965,characteristics_io_time
2,53.97501754760742,characteristics_job_time
3,1.0,characteristics_num_apps
4,1581.0,characteristics_num_files
...,...,...
145,15.239039059961215,timings_time__compute_hlm
146,2.7886829250492156,timings_time__compute_main_view
147,5.691171610960737,timings_time__compute_perspectives
148,2.4482154350262135,timings_time__detect_bottlenecks


time: 8.84 ms (started: 2024-01-15 19:46:52 -08:00)


In [34]:
dfT = df.set_index('key').T
dfT

key,characteristics_complexity,characteristics_io_time,characteristics_job_time,characteristics_num_apps,characteristics_num_files,characteristics_num_nodes,characteristics_num_ops,characteristics_num_procs,characteristics_num_time_periods,characteristics_per_io_time,...,throughputs_time__perspectives,throughputs_time__reasoned_records,throughputs_time__rules,throughputs_time__slope_filtered_records,timings_time__attach_reasons,timings_time__compute_hlm,timings_time__compute_main_view,timings_time__compute_perspectives,timings_time__detect_bottlenecks,timings_time__read_traces
15,7.94162558639499,1.988731704874965,53.97501754760742,1.0,1581.0,1.0,2478922.0,1024.0,54.0,0.036845410992610614,...,2.635661165288218,1243.145093113967,0.9403518102223654,27759.48623579292,6.380590684013441,15.239039059961215,2.7886829250492156,5.691171610960737,2.4482154350262135,0


time: 18.6 ms (started: 2024-01-15 20:18:31 -08:00)


In [36]:
con = sqlite3.connect("city_spec.db")

# drop data into database
dfT.to_sql("MyTable3", con, if_exists='append', index_label='name')

con.close()

time: 54.3 ms (started: 2024-01-15 20:46:39 -08:00)


In [42]:
from wisio.utils.dask_agg import unique

ddf = dd.from_pandas(df, npartitions=1)

ddf.groupby('15').agg({'key': unique()}).compute()

,key
15,
7.94162558639499,['characteristics_complexity']
1.988731704874965,['characteristics_io_time']
53.97501754760742,['characteristics_job_time']
1.0,"['characteristics_num_apps', 'counts_num_metri..."
1581.0,['characteristics_num_files']
...,...
6.380590684013441,['timings_time__attach_reasons']
15.239039059961215,['timings_time__compute_hlm']
2.7886829250492156,['timings_time__compute_main_view']


time: 443 ms (started: 2024-01-15 21:20:56 -08:00)


In [60]:
ddf.set_index('15')._meta.index.names

FrozenList(['15'])

time: 29.9 ms (started: 2024-01-15 21:37:52 -08:00)


In [56]:
ddf.set_index('15').index.map_partitions(lambda x: print(x.names)).compute()

['15']
[None]
['15']


0    None
dtype: object

time: 50 ms (started: 2024-01-15 21:36:52 -08:00)


In [75]:
'15' in df.set_index('key').loc['characteristics_complexity']

True

time: 4.2 ms (started: 2024-01-15 22:13:30 -08:00)


In [78]:
list(sorted(['abc']))

['abc']

time: 3.09 ms (started: 2024-01-15 22:29:07 -08:00)


In [96]:
list(sorted(df.groupby('15').agg({'key': set}).loc[['False']]['key']))

[{'setup_checkpoint', 'setup_debug', 'setup_verbose'}]

time: 9.25 ms (started: 2024-01-15 22:34:12 -08:00)
